In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Mnist

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
dataset = 'Mnist'
# dataset = 'IllusionAnimals'

df_raw = pd.read_csv(f'/content/drive/MyDrive/GPT-4_results/GPT-4_{dataset}_raw.csv')
df_illusion = pd.read_csv(f'/content/drive/MyDrive/GPT-4_results/GPT-4_{dataset}_illusion.csv')
df_illusionless = pd.read_csv(f'/content/drive/MyDrive/GPT-4_results/GPT-4_{dataset}_illusionless.csv')
df_illusion_filtered = pd.read_csv(f'/content/drive/MyDrive/GPT-4_results/GPT-4_{dataset}_illusion_filtered.csv')
df_illusionless_filtered = pd.read_csv(f'/content/drive/MyDrive/GPT-4_results/GPT-4_{dataset}_illusionless_filtered.csv')


In [ ]:
df_raw

,Unnamed: 0,image_name,Pprompt,Nprompt,illusion_strength,label,gpt-4o
0,0,Mnist_1,A field of blooming sunflowers swaying in the ...,low quality,1.5,7,digit 7
1,1,Mnist_2,A peaceful countryside scene with grazing shee...,low quality,1.5,2,error
2,2,Mnist_3,A tranquil pond with lily pads floating on the...,low quality,1.5,1,error
3,3,Mnist_4,A sunny vineyard with rows of ripe grapes,low quality,1.5,0,error
4,4,Mnist_5,A picturesque vineyard at sunset with the sky ...,low quality,1.5,4,error
...,...,...,...,...,...,...,...
1104,1104,Mnist_1105,Misty jungle surrounded by vibrant flowers and...,low quality,1.5,8,error
1105,1105,Mnist_1106,A forest with blooming flowers,low quality,1.5,0,error
1106,1106,Mnist_1107,"Desolate desert landscape, shifting sands illu...",low quality,1.5,5,error
1107,1107,Mnist_1108,A vast desert with a towering canyon in the di...,low quality,1.5,0,Digit 0


In [ ]:
import torch
import numpy as np
import random

# Make reproducible code
GLOBAL_SEED = 10

np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
torch.use_deterministic_algorithms(True)
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# illusionless_indexes = random.sample(range(len(df_illusion)), int(len(df_illusion)*0.1))

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


In [ ]:
raw_classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' ],
}

classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'No illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'No illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster', 'No illusion' ],
}

# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_ill_images'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_illusion_images_filtered'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_illusionless_images'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_illusionless_images_filtered'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_raw_images'].unique()}.issubset({item.strip().lower() for item in raw_classes[dataset]})

In [ ]:
{item.strip().replace('\'', '').lower() for item in df_raw[f'gpt-4o'].unique()}

{'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9',
 'error'}

In [ ]:
{item.strip().replace('\'', '').replace('‘','').replace('’','').lower() for item in df_illusion[f'gpt-4o'].unique()}

{'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9',
 'error, you uploaded an unsupported image. please make sure your image is below 20 mb in size and is of one the following formats: [png, jpeg, gif, webp].',
 'no illusion'}

In [ ]:
{item.strip().replace('\'', '').lower() for item in df_illusionless[f'gpt-4o'].unique()}

{'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 6',
 'digit 7',
 'digit 8',
 'error, you uploaded an unsupported image. please make sure your image is below 20 mb in size and is of one the following formats: [png, jpeg, gif, webp].',
 'no illusion'}

In [ ]:
{item.strip().replace('\'', '').lower() for item in df_illusion_filtered[f'gpt-4o'].unique()}

{'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9',
 'error, the server had an error processing your request. sorry about that! you can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (please include the request id req_14019500bf362a4b2e7f24b3644e7175 in your email.)',
 'error, the server had an error processing your request. sorry about that! you can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (please include the request id req_c5768bd293b4dcd6496557c002db3073 in your email.)',
 'error, you uploaded an unsupported image. please make sure your image is below 20 mb in size and is of one the following formats: [png, jpeg, gif, webp].',
 'error, your input image may contain content that is not allowed by our safety system.',
 'no illusion'}

In [ ]:
{item.strip().replace('\'', '').lower() for item in df_illusionless_filtered[f'gpt-4o'].unique()}

{'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9',
 'no illusion'}

In [ ]:
raw_classes[dataset]

['digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9']

## Raw

In [ ]:
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
y_true = df_raw['label'].apply(lambda x: idx2label[x].strip().lower())
y_pred = df_raw[f'gpt-4o'].apply(lambda x: x.strip().replace('\'', '').lower())

In [ ]:
filtered_df = df_raw[~y_pred.str.contains('error')]
filtered_y_true = y_true[~y_pred.str.contains('error')]
filtered_y_pred = y_pred[~y_pred.str.contains('error')]

In [ ]:
filtered_y_pred

0       digit 7
5       digit 1
14      digit 1
34      digit 7
37      digit 1
         ...   
1086    digit 5
1087    digit 5
1093    digit 0
1101    digit 1
1107    digit 0
Name: gpt-4o, Length: 247, dtype: object

In [ ]:
filtered_y_true

0       digit 7
5       digit 1
14      digit 1
34      digit 7
37      digit 1
         ...   
1086    digit 5
1087    digit 5
1093    digit 0
1101    digit 6
1107    digit 0
Name: label, Length: 247, dtype: object

In [ ]:
set(list(filtered_y_pred))

{'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9'}

In [ ]:
set(list(filtered_y_true))

{'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9'}

In [ ]:
coverage = len(filtered_y_pred)/len(y_true)
accuracy = accuracy_score(filtered_y_true, filtered_y_pred)
precision = precision_score(filtered_y_true, filtered_y_pred, average='macro')
recall = recall_score(filtered_y_true, filtered_y_pred, average='macro')
f1 = f1_score(filtered_y_true, filtered_y_pred, average='macro')

print("Coverage:", coverage)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Coverage: 0.22272317403065825
Accuracy: 0.8987854251012146
Precision: 0.9026076140171885
Recall: 0.8589381347082714
F1 score: 0.8698863566230989


In [ ]:
print("Coverage:", round(coverage*100,2))
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Coverage: 22.27
Accuracy: 89.88
Precision: 90.26
Recall: 85.89
F1 score: 86.99


## Illusion

In [ ]:
classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'no illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'no illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' , 'no illusion'],
}
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
y_true = list(df_illusion['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df_illusionless['label'].apply(lambda x: 'no illusion'))

y_pred = list(df_illusion[f'gpt-4o'].apply(lambda x: x.strip().replace('\'', '').replace('‘','').replace('’','').lower())) + list(df_illusionless[f'gpt-4o'].apply(lambda x: x.strip().replace('\'', '').lower()))

In [ ]:
y_true_illusion = list(df_illusion['label'].apply(lambda x: idx2label[x].strip().lower()))
y_true_illusionless = list(df_illusionless['label'].apply(lambda x: 'no illusion'))
y_pred_illusion = list(df_illusion[f'gpt-4o'].apply(lambda x: x.strip().replace('\'', '').replace('‘','').replace('’','').lower()))
y_pred_illusionless = list(df_illusionless[f'gpt-4o'].apply(lambda x: x.strip().replace('\'', '').lower()))

In [ ]:
set(y_pred_illusion)

{'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9',
 'error, you uploaded an unsupported image. please make sure your image is below 20 mb in size and is of one the following formats: [png, jpeg, gif, webp].',
 'no illusion'}

In [ ]:
filtered_indices_illusion = [i for i, pred in enumerate(y_pred_illusion) if 'error' in pred]
y_pred_illusion_filtered = [pred for i, pred in enumerate(y_pred_illusion) if i not in filtered_indices_illusion]
y_true_illusion_filtered = [true for i, true in enumerate(y_true_illusion) if i not in filtered_indices_illusion]

filtered_indices_illusionless = [i for i, pred in enumerate(y_pred_illusionless) if 'error' in pred]
y_pred_illusionless_filtered = [pred for i, pred in enumerate(y_pred_illusionless) if i not in filtered_indices_illusionless]
y_true_illusionless_filtered = [true for i, true in enumerate(y_true_illusionless) if i not in filtered_indices_illusionless]

In [ ]:
whole_y_true = y_true_illusion_filtered + y_true_illusionless_filtered
whole_y_pred = y_pred_illusion_filtered + y_pred_illusionless_filtered

In [ ]:
coverage = len(whole_y_true)/(len(y_true_illusion)+len(y_true_illusionless))
accuracy = accuracy_score(whole_y_true, whole_y_pred)
precision = precision_score(whole_y_true, whole_y_pred, average='macro')
recall = recall_score(whole_y_true, whole_y_pred, average='macro')
f1 = f1_score(whole_y_true, whole_y_pred, average='macro')

print("Coverage:", coverage)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Coverage: 0.9983593109105825
Accuracy: 0.37469186524239934
Precision: 0.4494363843370843
Recall: 0.37397266377310207
F1 score: 0.36953191320413925


In [ ]:
print("Coverage:", round(coverage*100,2))
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Coverage: 99.84
Accuracy: 37.47
Precision: 44.94
Recall: 37.4
F1 score: 36.95


## Illusion Filtered

In [ ]:
classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'no illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'no illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' , 'no illusion'],
}
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
# y_true = list(df_illusion_filtered['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df_illusionless_filtered.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))


# y_pred = list(df[f'illusion_images_filtered'].apply(lambda x: x.strip().replace('\'', '').lower())) + list(df.filter(items=illusionless_indexes, axis=0)[f'illusionless_images_filtered'].apply(lambda x: x.strip().replace('\'', '').lower()))

In [ ]:
y_true_illusion_f = list(df_illusion_filtered['label'].apply(lambda x: idx2label[x].strip().lower()))
y_true_illusionless_f = list(df_illusionless_filtered['label'].apply(lambda x: 'no illusion'))
y_pred_illusion_f = list(df_illusion_filtered[f'gpt-4o'].apply(lambda x: x.strip().replace('\'', '').replace('‘','').replace('’','').lower()))
y_pred_illusionless_f = list(df_illusionless_filtered[f'gpt-4o'].apply(lambda x: x.strip().replace('\'', '').lower()))

In [ ]:
filtered_indices_illusion_filtered = [i for i, pred in enumerate(y_pred_illusion_f) if 'error' in pred]
y_pred_illusion_f_filtered = [pred for i, pred in enumerate(y_pred_illusion_f) if i not in filtered_indices_illusion_filtered]
y_true_illusion_f_filtered = [true for i, true in enumerate(y_true_illusion_f) if i not in filtered_indices_illusion_filtered]

filtered_indices_illusionless_filtered = [i for i, pred in enumerate(y_pred_illusionless_f) if 'error' in pred]
y_pred_illusionless_f_filtered = [pred for i, pred in enumerate(y_pred_illusionless_f) if i not in filtered_indices_illusionless_filtered]
y_true_illusionless_f_filtered = [true for i, true in enumerate(y_true_illusionless_f) if i not in filtered_indices_illusionless_filtered]

In [ ]:
whole_y_true = y_true_illusion_f_filtered + y_true_illusionless_f_filtered
whole_y_pred = y_pred_illusion_f_filtered + y_pred_illusionless_f_filtered

In [ ]:
coverage = len(whole_y_true)/(len(y_true_illusion_f)+len(y_true_illusionless_f))
accuracy = accuracy_score(whole_y_true, whole_y_pred)
precision = precision_score(whole_y_true, whole_y_pred, average='macro')
recall = recall_score(whole_y_true, whole_y_pred, average='macro')
f1 = f1_score(whole_y_true, whole_y_pred, average='macro')

print("Coverage:", coverage)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Coverage: 0.9680065627563577
Accuracy: 0.6796610169491526
Precision: 0.7039038335822556
Recall: 0.6798719438482809
F1 score: 0.6832106056176457


In [ ]:
print("Coverage:", round(coverage*100,2))
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Coverage: 96.8
Accuracy: 67.97
Precision: 70.39
Recall: 67.99
F1 score: 68.32


# FashionMnist

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
dataset = 'FashionMnist'
# dataset = 'IllusionAnimals'

df_raw = pd.read_csv(f'/content/drive/MyDrive/GPT-4_results/GPT-4_{dataset}_raw.csv')
df_illusion = pd.read_csv(f'/content/drive/MyDrive/GPT-4_results/GPT-4_{dataset}_illusion.csv')
df_illusionless = pd.read_csv(f'/content/drive/MyDrive/GPT-4_results/GPT-4_{dataset}_illusionless.csv')
df_illusion_filtered = pd.read_csv(f'/content/drive/MyDrive/GPT-4_results/GPT-4_{dataset}_illusion_filtered.csv')
df_illusionless_filtered = pd.read_csv(f'/content/drive/MyDrive/GPT-4_results/GPT-4_{dataset}_illusionless_filtered.csv')


In [ ]:
df_raw

,Unnamed: 0,image_name,Pprompt,Nprompt,illusion_strength,label,gpt-4o
0,0,FashionMnist_1,Two friends hiking on a mountain during sunset,low quality,1.5,9,Sneaker
1,1,FashionMnist_2,A starry night sky with constellations glowing,low quality,1.5,2,Pullover
2,2,FashionMnist_3,A Thanksgiving feast giving thanks and love,low quality,1.5,1,error
3,3,FashionMnist_4,A gecko on a large leaf in a tropical rainforest,low quality,1.5,1,error
4,4,FashionMnist_5,A Hanukkah menorah lighting symbolizing faith,low quality,1.5,6,error
...,...,...,...,...,...,...,...
1147,1147,FashionMnist_1148,"Electrified waterfalls cascade, neon rivers re...",low quality,1.5,2,'Pullover'
1148,1148,FashionMnist_1149,A rocky desert landscape with cacti and tumble...,low quality,1.5,2,Pullover
1149,1149,FashionMnist_1150,A majestic waterfall plunging into a crystal-c...,low quality,1.5,5,'Sandal'
1150,1150,FashionMnist_1151,"Eerie silence surrounds snowy mountains, glowi...",low quality,1.5,3,"error, Your input image may contain content th..."


In [ ]:
import torch
import numpy as np
import random

# Make reproducible code
GLOBAL_SEED = 10

np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
torch.use_deterministic_algorithms(True)
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# illusionless_indexes = random.sample(range(len(df_illusion)), int(len(df_illusion)*0.1))

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


In [ ]:
raw_classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' ],
}

classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'No illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'No illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster', 'No illusion' ],
}

# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_ill_images'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_illusion_images_filtered'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_illusionless_images'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_illusionless_images_filtered'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_raw_images'].unique()}.issubset({item.strip().lower() for item in raw_classes[dataset]})

In [ ]:
{item.strip().replace('\'', '').lower() for item in df_raw[f'gpt-4o'].unique()}

{'ankle boot',
 'ankle boot.',
 'bag',
 'coat',
 'dress',
 'error',
 'error, the server had an error processing your request. sorry about that! you can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (please include the request id req_27c8625ce826d4865451267af46d0aba in your email.)',
 'error, your input image may contain content that is not allowed by our safety system.',
 'pullover',
 'pullover.',
 'sandal',
 'shirt',
 'sneaker',
 'sneaker.',
 't-shirt/top',
 'trouser'}

In [ ]:
{item.strip().replace('\'', '').replace('‘','').replace('’','').lower() for item in df_illusion[f'gpt-4o'].unique()}

{'ankle boot',
 'bag',
 'coat',
 'dress',
 'error, you uploaded an unsupported image. please make sure your image is below 20 mb in size and is of one the following formats: [png, jpeg, gif, webp].',
 'no illusion',
 'no illusion.',
 'pullover',
 'sandal',
 'shirt',
 'sneaker',
 't-shirt/top',
 'trouser'}

In [ ]:
{item.strip().replace('\'', '').lower() for item in df_illusionless[f'gpt-4o'].unique()}

{'dress', 'no illusion', 't-shirt/top'}

In [ ]:
{item.strip().replace('\'', '').lower() for item in df_illusion_filtered[f'gpt-4o'].unique()}

{'ankle boot',
 'bag',
 'coat',
 'dress',
 'error, the server had an error processing your request. sorry about that! you can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (please include the request id req_4a146828222c77f5dd47997eadb05340 in your email.)',
 'error, the server had an error processing your request. sorry about that! you can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (please include the request id req_a221d634884e4497697e39ed3f12b753 in your email.)',
 'error, you uploaded an unsupported image. please make sure your image is below 20 mb in size and is of one the following formats: [png, jpeg, gif, webp].',
 'error, your input image may contain content that is not allowed by our safety system.',
 'no illusion',
 'puller',
 'pullover',
 'sandal',
 'shirt',
 'sneaker',
 't-shirt/top',
 'trouser'}

In [ ]:
{item.strip().replace('\'', '').lower() for item in df_illusionless_filtered[f'gpt-4o'].unique()}

{'ankle boot',
 'bag',
 'dress',
 'no illusion',
 'pullover',
 't-shirt/top',
 'trouser'}

In [ ]:
raw_classes[dataset]

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

## Raw

In [ ]:
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
y_true = df_raw['label'].apply(lambda x: idx2label[x].strip().lower())
y_pred = df_raw[f'gpt-4o'].apply(lambda x: x.strip().replace('\'', '').replace('.', '').lower())

In [ ]:
set(list(y_pred))

{'ankle boot',
 'bag',
 'coat',
 'dress',
 'error',
 'error, the server had an error processing your request sorry about that! you can retry your request, or contact us through our help center at helpopenaicom if you keep seeing this error (please include the request id req_27c8625ce826d4865451267af46d0aba in your email)',
 'error, your input image may contain content that is not allowed by our safety system',
 'pullover',
 'sandal',
 'shirt',
 'sneaker',
 't-shirt/top',
 'trouser'}

In [ ]:
filtered_df = df_raw[~y_pred.str.contains('error')]
filtered_y_true = y_true[~y_pred.str.contains('error')]
filtered_y_pred = y_pred[~y_pred.str.contains('error')]

In [ ]:
set(filtered_y_pred)

{'ankle boot',
 'bag',
 'coat',
 'dress',
 'pullover',
 'sandal',
 'shirt',
 'sneaker',
 't-shirt/top',
 'trouser'}

In [ ]:
filtered_y_true

0       digit 7
5       digit 1
14      digit 1
34      digit 7
37      digit 1
         ...   
1086    digit 5
1087    digit 5
1093    digit 0
1101    digit 6
1107    digit 0
Name: label, Length: 247, dtype: object

In [ ]:
set(list(filtered_y_pred))

{'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9'}

In [ ]:
set(list(filtered_y_true))

{'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9'}

In [ ]:
coverage = len(filtered_y_pred)/len(y_true)
accuracy = accuracy_score(filtered_y_true, filtered_y_pred)
precision = precision_score(filtered_y_true, filtered_y_pred, average='macro')
recall = recall_score(filtered_y_true, filtered_y_pred, average='macro')
f1 = f1_score(filtered_y_true, filtered_y_pred, average='macro')

print("Coverage:", coverage)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Coverage: 0.7074652777777778
Accuracy: 0.6711656441717792
Precision: 0.7105354859806012
Recall: 0.6788639531473122
F1 score: 0.659301091821997


In [ ]:
print("Coverage:", round(coverage*100,2))
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Coverage: 70.75
Accuracy: 67.12
Precision: 71.05
Recall: 67.89
F1 score: 65.93


## Illusion

In [ ]:
classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'no illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'no illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' , 'no illusion'],
}
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
y_true = list(df_illusion['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df_illusionless['label'].apply(lambda x: 'no illusion'))

y_pred = list(df_illusion[f'gpt-4o'].apply(lambda x: x.strip().replace('\'', '').replace('‘','').replace('’','').lower())) + list(df_illusionless[f'gpt-4o'].apply(lambda x: x.strip().replace('\'', '').lower()))

In [ ]:
y_true_illusion = list(df_illusion['label'].apply(lambda x: idx2label[x].strip().lower()))
y_true_illusionless = list(df_illusionless['label'].apply(lambda x: 'no illusion'))
y_pred_illusion = list(df_illusion[f'gpt-4o'].apply(lambda x: x.strip().replace('\'', '').replace('‘','').replace('’','').replace('.','').lower()))
y_pred_illusionless = list(df_illusionless[f'gpt-4o'].apply(lambda x: x.strip().replace('\'', '').lower()))

In [ ]:
set(y_pred_illusion)

{'ankle boot',
 'bag',
 'coat',
 'dress',
 'error, you uploaded an unsupported image please make sure your image is below 20 mb in size and is of one the following formats: [png, jpeg, gif, webp]',
 'no illusion',
 'pullover',
 'sandal',
 'shirt',
 'sneaker',
 't-shirt/top',
 'trouser'}

In [ ]:
filtered_indices_illusion = [i for i, pred in enumerate(y_pred_illusion) if 'error' in pred]
y_pred_illusion_filtered = [pred for i, pred in enumerate(y_pred_illusion) if i not in filtered_indices_illusion]
y_true_illusion_filtered = [true for i, true in enumerate(y_true_illusion) if i not in filtered_indices_illusion]

filtered_indices_illusionless = [i for i, pred in enumerate(y_pred_illusionless) if 'error' in pred]
y_pred_illusionless_filtered = [pred for i, pred in enumerate(y_pred_illusionless) if i not in filtered_indices_illusionless]
y_true_illusionless_filtered = [true for i, true in enumerate(y_true_illusionless) if i not in filtered_indices_illusionless]

In [ ]:
whole_y_true = y_true_illusion_filtered + y_true_illusionless_filtered
whole_y_pred = y_pred_illusion_filtered + y_pred_illusionless_filtered

In [ ]:
coverage = len(whole_y_true)/(len(y_true_illusion)+len(y_true_illusionless))
accuracy = accuracy_score(whole_y_true, whole_y_pred)
precision = precision_score(whole_y_true, whole_y_pred, average='macro')
recall = recall_score(whole_y_true, whole_y_pred, average='macro')
f1 = f1_score(whole_y_true, whole_y_pred, average='macro')

print("Coverage:", coverage)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Coverage: 0.9992107340173638
Accuracy: 0.16666666666666666
Precision: 0.27762110515297295
Recall: 0.16609674370633892
F1 score: 0.12718988129724898


In [ ]:
print("Coverage:", round(coverage*100,2))
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Coverage: 99.92
Accuracy: 16.67
Precision: 27.76
Recall: 16.61
F1 score: 12.72


## Illusion Filtered

In [ ]:
classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'no illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'no illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' , 'no illusion'],
}
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
# y_true = list(df_illusion_filtered['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df_illusionless_filtered.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))


# y_pred = list(df[f'illusion_images_filtered'].apply(lambda x: x.strip().replace('\'', '').lower())) + list(df.filter(items=illusionless_indexes, axis=0)[f'illusionless_images_filtered'].apply(lambda x: x.strip().replace('\'', '').lower()))

In [ ]:
y_true_illusion_f = list(df_illusion_filtered['label'].apply(lambda x: idx2label[x].strip().lower()))
y_true_illusionless_f = list(df_illusionless_filtered['label'].apply(lambda x: 'no illusion'))
y_pred_illusion_f = list(df_illusion_filtered[f'gpt-4o'].apply(lambda x: x.strip().replace('\'', '').replace('‘','').replace('’','').lower()))
y_pred_illusionless_f = list(df_illusionless_filtered[f'gpt-4o'].apply(lambda x: x.strip().replace('\'', '').lower()))

In [ ]:
set(y_pred_illusionless_f)

{'ankle boot',
 'bag',
 'dress',
 'no illusion',
 'pullover',
 't-shirt/top',
 'trouser'}

In [ ]:
filtered_indices_illusion_filtered = [i for i, pred in enumerate(y_pred_illusion_f) if 'error' in pred]
y_pred_illusion_f_filtered = [pred for i, pred in enumerate(y_pred_illusion_f) if i not in filtered_indices_illusion_filtered]
y_true_illusion_f_filtered = [true for i, true in enumerate(y_true_illusion_f) if i not in filtered_indices_illusion_filtered]

filtered_indices_illusionless_filtered = [i for i, pred in enumerate(y_pred_illusionless_f) if 'error' in pred]
y_pred_illusionless_f_filtered = [pred for i, pred in enumerate(y_pred_illusionless_f) if i not in filtered_indices_illusionless_filtered]
y_true_illusionless_f_filtered = [true for i, true in enumerate(y_true_illusionless_f) if i not in filtered_indices_illusionless_filtered]

In [ ]:
whole_y_true = y_true_illusion_f_filtered + y_true_illusionless_f_filtered
whole_y_pred = y_pred_illusion_f_filtered + y_pred_illusionless_f_filtered

In [ ]:
coverage = len(whole_y_true)/(len(y_true_illusion_f)+len(y_true_illusionless_f))
accuracy = accuracy_score(whole_y_true, whole_y_pred)
precision = precision_score(whole_y_true, whole_y_pred, average='macro')
recall = recall_score(whole_y_true, whole_y_pred, average='macro')
f1 = f1_score(whole_y_true, whole_y_pred, average='macro')

print("Coverage:", coverage)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Coverage: 0.994475138121547
Accuracy: 0.4873015873015873
Precision: 0.48761683776395076
Recall: 0.4472995225288339
F1 score: 0.41583784066681445


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print("Coverage:", round(coverage*100,2))
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Coverage: 99.45
Accuracy: 48.73
Precision: 48.76
Recall: 44.73
F1 score: 41.58


# IllusionAnimals

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
dataset = 'IllusionAnimals'
# dataset = 'IllusionAnimals'

df_raw = pd.read_csv(f'/content/drive/MyDrive/GPT-4_results/GPT-4_{dataset}_raw.csv')
df_illusion = pd.read_csv(f'/content/drive/MyDrive/GPT-4_results/GPT-4_{dataset}_illusion.csv')
df_illusionless = pd.read_csv(f'/content/drive/MyDrive/GPT-4_results/GPT-4_{dataset}_illusionless.csv')
df_illusion_filtered = pd.read_csv(f'/content/drive/MyDrive/GPT-4_results/GPT-4_{dataset}_illusion_filtered.csv')
df_illusionless_filtered = pd.read_csv(f'/content/drive/MyDrive/GPT-4_results/GPT-4_{dataset}_illusionless_filtered.csv')


In [ ]:
df_raw

,Unnamed: 0,image_name,Pprompt,Nprompt,illusion_strength,label,gpt-4o
0,0,IllusionAnimals_1,A raging river flowing through a dense jungle ...,low quality,2.5,cat,cat
1,1,IllusionAnimals_2,A starry night sky over a tranquil lake,low quality,2.5,cat,Cat
2,2,IllusionAnimals_3,Jaguar (Panthera Onca) patrolling South Americ...,low quality,2.5,cat,Cat
3,3,IllusionAnimals_4,"Flowing lava illuminates cavern walls, ancient...",low quality,2.5,cat,cat
4,4,IllusionAnimals_5,"Dense forest canopy, sunlight filters through,...",low quality,2.5,cat,cat
...,...,...,...,...,...,...,...
995,995,IllusionAnimals_996,Saharan sandstorm turning daytime into darkness,low quality,2.5,rooster,Rooster
996,996,IllusionAnimals_997,Urban city with skyscrapers and traffic,low quality,2.5,rooster,Rooster
997,997,IllusionAnimals_998,A serene meadow with wildflowers and butterflies,low quality,2.5,rooster,rooster
998,998,IllusionAnimals_999,Elk bugling in crisp autumn air,low quality,2.5,rooster,Rooster


In [ ]:
import torch
import numpy as np
import random

# Make reproducible code
GLOBAL_SEED = 10

np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
torch.use_deterministic_algorithms(True)
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# illusionless_indexes = random.sample(range(len(df_illusion)), int(len(df_illusion)*0.1))

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


In [ ]:
raw_classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' ],
}

classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'No illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'No illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster', 'No illusion' ],
}

# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_ill_images'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_illusion_images_filtered'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_illusionless_images'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_illusionless_images_filtered'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_raw_images'].unique()}.issubset({item.strip().lower() for item in raw_classes[dataset]})

In [ ]:
{item.strip().replace('\'', '').lower() for item in df_raw[f'gpt-4o'].unique()}

{'butterfly',
 'butterfly.',
 'cat',
 'cat.',
 'deer',
 'dog',
 'dog.',
 'elephant',
 'elephant.',
 'error, you uploaded an unsupported image. please make sure your image is below 20 mb in size and is of one the following formats: [png, jpeg, gif, webp].',
 'fish',
 'horse',
 'horse.',
 'pigeon',
 'pigeon.',
 'rooster',
 'rooster.',
 'snake'}

In [ ]:
{item.strip().replace('\'', '').replace('‘','').replace('’','').lower() for item in df_illusion[f'gpt-4o'].unique()}

{'bear',
 'bird',
 'bird,',
 'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'error, the server had an error processing your request. sorry about that! you can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (please include the request id req_59a745199938d2ea6c8dee5090d7893a in your email.)',
 'error, the server had an error processing your request. sorry about that! you can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (please include the request id req_d85499d1e3e6cbf42c8ca0d318831d3c in your email.)',
 'error, you uploaded an unsupported image. please make sure your image is below 20 mb in size and is of one the following formats: [png, jpeg, gif, webp].',
 'fish',
 'frog',
 'horse',
 'no illusion',
 'pigeon',
 'rooster',
 'snake'}

In [ ]:
{item.strip().replace('\'', '').lower() for item in df_illusionless[f'gpt-4o'].unique()}

{'butterfly',
 'cat',
 'deer',
 'elephant',
 'error, the server had an error processing your request. sorry about that! you can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (please include the request id req_a68394e2402c1fcce8365e8d992a4110 in your email.)',
 'fish',
 'horse',
 'no illusion',
 'pigeon',
 'snake'}

In [ ]:
{item.strip().replace('\'', '').lower() for item in df_illusion_filtered[f'gpt-4o'].unique()}

{'bird',
 'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'error, you uploaded an unsupported image. please make sure your image is below 20 mb in size and is of one the following formats: [png, jpeg, gif, webp].',
 'fish',
 'horse',
 'no illusion',
 'pigeon',
 'rooster',
 'snake'}

In [ ]:
{item.strip().replace('\'', '').lower() for item in df_illusionless_filtered[f'gpt-4o'].unique()}

{'bear',
 'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'fish',
 'horse',
 'no illusion',
 'pigeon',
 'snake'}

In [ ]:
raw_classes[dataset]

['cat',
 'dog',
 'pigeon',
 'butterfly',
 'elephant',
 'horse',
 'deer',
 'snake',
 'fish',
 'rooster']

## Raw

In [ ]:
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
y_true = df_raw['label'].apply(lambda x: x.strip().lower())
y_pred = df_raw[f'gpt-4o'].apply(lambda x: x.strip().replace('\'', '').replace('.', '').lower())

In [ ]:
set(list(y_pred))

{'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'error, you uploaded an unsupported image please make sure your image is below 20 mb in size and is of one the following formats: [png, jpeg, gif, webp]',
 'fish',
 'horse',
 'pigeon',
 'rooster',
 'snake'}

In [ ]:
set(y_true)

{'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'fish',
 'horse',
 'pigeon',
 'rooster',
 'snake'}

In [ ]:
filtered_df = df_raw[~y_pred.str.contains('error')]
filtered_y_true = y_true[~y_pred.str.contains('error')]
filtered_y_pred = y_pred[~y_pred.str.contains('error')]

In [ ]:
set(filtered_y_pred)

{'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'fish',
 'horse',
 'pigeon',
 'rooster',
 'snake'}

In [ ]:
filtered_y_true

0          cat
1          cat
2          cat
3          cat
4          cat
        ...   
995    rooster
996    rooster
997    rooster
998    rooster
999    rooster
Name: label, Length: 999, dtype: object

In [ ]:
set(list(filtered_y_pred))

{'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'fish',
 'horse',
 'pigeon',
 'rooster',
 'snake'}

In [ ]:
set(list(filtered_y_true))

{'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'fish',
 'horse',
 'pigeon',
 'rooster',
 'snake'}

In [ ]:
coverage = len(filtered_y_pred)/len(y_true)
accuracy = accuracy_score(filtered_y_true, filtered_y_pred)
precision = precision_score(filtered_y_true, filtered_y_pred, average='macro')
recall = recall_score(filtered_y_true, filtered_y_pred, average='macro')
f1 = f1_score(filtered_y_true, filtered_y_pred, average='macro')

print("Coverage:", coverage)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Coverage: 0.999
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 score: 1.0


In [ ]:
print("Coverage:", round(coverage*100,2))
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Coverage: 99.9
Accuracy: 100.0
Precision: 100.0
Recall: 100.0
F1 score: 100.0


## Illusion

In [ ]:
classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'no illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'no illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' , 'no illusion'],
}
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
y_true = list(df_illusion['label'].apply(lambda x: x.strip().lower())) + list(df_illusionless['label'].apply(lambda x: 'no illusion'))

y_pred = list(df_illusion[f'gpt-4o'].apply(lambda x: x.strip().replace('\'', '').replace('‘','').replace('’','').replace(',', '').lower())) + list(df_illusionless[f'gpt-4o'].apply(lambda x: x.strip().replace('\'', '').lower()))

In [ ]:
set(y_pred)

{'bear',
 'bird',
 'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'error the server had an error processing your request. sorry about that! you can retry your request or contact us through our help center at help.openai.com if you keep seeing this error. (please include the request id req_59a745199938d2ea6c8dee5090d7893a in your email.)',
 'error the server had an error processing your request. sorry about that! you can retry your request or contact us through our help center at help.openai.com if you keep seeing this error. (please include the request id req_d85499d1e3e6cbf42c8ca0d318831d3c in your email.)',
 'error you uploaded an unsupported image. please make sure your image is below 20 mb in size and is of one the following formats: [png jpeg gif webp].',
 'error, the server had an error processing your request. sorry about that! you can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (please include the request id r

In [ ]:
y_true_illusion = list(df_illusion['label'].apply(lambda x: x.strip().lower()))
y_true_illusionless = list(df_illusionless['label'].apply(lambda x: 'no illusion'))
y_pred_illusion = list(df_illusion[f'gpt-4o'].apply(lambda x: x.strip().replace('\'', '').replace('‘','').replace('’','').replace('.','').replace(',','').lower()))
y_pred_illusionless = list(df_illusionless[f'gpt-4o'].apply(lambda x: x.strip().replace('\'', '').lower()))

In [ ]:
set(y_pred_illusionless)

{'butterfly',
 'cat',
 'deer',
 'elephant',
 'error, the server had an error processing your request. sorry about that! you can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (please include the request id req_a68394e2402c1fcce8365e8d992a4110 in your email.)',
 'fish',
 'horse',
 'no illusion',
 'pigeon',
 'snake'}

In [ ]:
filtered_indices_illusion = [i for i, pred in enumerate(y_pred_illusion) if 'error' in pred]
y_pred_illusion_filtered = [pred for i, pred in enumerate(y_pred_illusion) if i not in filtered_indices_illusion]
y_true_illusion_filtered = [true for i, true in enumerate(y_true_illusion) if i not in filtered_indices_illusion]

filtered_indices_illusionless = [i for i, pred in enumerate(y_pred_illusionless) if 'error' in pred]
y_pred_illusionless_filtered = [pred for i, pred in enumerate(y_pred_illusionless) if i not in filtered_indices_illusionless]
y_true_illusionless_filtered = [true for i, true in enumerate(y_true_illusionless) if i not in filtered_indices_illusionless]

In [ ]:
set(y_pred_illusionless_filtered)

{'butterfly',
 'cat',
 'deer',
 'elephant',
 'fish',
 'horse',
 'no illusion',
 'pigeon',
 'snake'}

In [ ]:
whole_y_true = y_true_illusion_filtered + y_true_illusionless_filtered
whole_y_pred = y_pred_illusion_filtered + y_pred_illusionless_filtered

In [ ]:
coverage = len(whole_y_true)/(len(y_true_illusion)+len(y_true_illusionless))
accuracy = accuracy_score(whole_y_true, whole_y_pred)
precision = precision_score(whole_y_true, whole_y_pred, average='macro')
recall = recall_score(whole_y_true, whole_y_pred, average='macro')
f1 = f1_score(whole_y_true, whole_y_pred, average='macro')

print("Coverage:", coverage)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Coverage: 0.9954545454545455
Accuracy: 0.34885844748858447
Precision: 0.3588741424943717
Recall: 0.27445165945165945
F1 score: 0.27077113097859773


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print("Coverage:", round(coverage*100,2))
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Coverage: 99.55
Accuracy: 34.89
Precision: 35.89
Recall: 27.45
F1 score: 27.08


## Illusion Filtered

In [ ]:
classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'no illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'no illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' , 'no illusion'],
}
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
# y_true = list(df_illusion_filtered['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df_illusionless_filtered.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))


# y_pred = list(df[f'illusion_images_filtered'].apply(lambda x: x.strip().replace('\'', '').lower())) + list(df.filter(items=illusionless_indexes, axis=0)[f'illusionless_images_filtered'].apply(lambda x: x.strip().replace('\'', '').lower()))

In [ ]:
y_true_illusion_f = list(df_illusion_filtered['label'].apply(lambda x: x.strip().lower()))
y_true_illusionless_f = list(df_illusionless_filtered['label'].apply(lambda x: 'no illusion'))
y_pred_illusion_f = list(df_illusion_filtered[f'gpt-4o'].apply(lambda x: x.strip().replace('\'', '').replace('‘','').replace('’','').lower()))
y_pred_illusionless_f = list(df_illusionless_filtered[f'gpt-4o'].apply(lambda x: x.strip().replace('\'', '').lower()))

In [ ]:
set(y_pred_illusion_f)

{'bird',
 'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'error, you uploaded an unsupported image. please make sure your image is below 20 mb in size and is of one the following formats: [png, jpeg, gif, webp].',
 'fish',
 'horse',
 'no illusion',
 'pigeon',
 'rooster',
 'snake'}

In [ ]:
filtered_indices_illusion_filtered = [i for i, pred in enumerate(y_pred_illusion_f) if 'error' in pred]
y_pred_illusion_f_filtered = [pred for i, pred in enumerate(y_pred_illusion_f) if i not in filtered_indices_illusion_filtered]
y_true_illusion_f_filtered = [true for i, true in enumerate(y_true_illusion_f) if i not in filtered_indices_illusion_filtered]

filtered_indices_illusionless_filtered = [i for i, pred in enumerate(y_pred_illusionless_f) if 'error' in pred]
y_pred_illusionless_f_filtered = [pred for i, pred in enumerate(y_pred_illusionless_f) if i not in filtered_indices_illusionless_filtered]
y_true_illusionless_f_filtered = [true for i, true in enumerate(y_true_illusionless_f) if i not in filtered_indices_illusionless_filtered]

In [ ]:
set(y_pred_illusion_f_filtered)

{'bird',
 'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'fish',
 'horse',
 'no illusion',
 'pigeon',
 'rooster',
 'snake'}

In [ ]:
whole_y_true = y_true_illusion_f_filtered + y_true_illusionless_f_filtered
whole_y_pred = y_pred_illusion_f_filtered + y_pred_illusionless_f_filtered

In [ ]:
coverage = len(whole_y_true)/(len(y_true_illusion_f)+len(y_true_illusionless_f))
accuracy = accuracy_score(whole_y_true, whole_y_pred)
precision = precision_score(whole_y_true, whole_y_pred, average='macro')
recall = recall_score(whole_y_true, whole_y_pred, average='macro')
f1 = f1_score(whole_y_true, whole_y_pred, average='macro')

print("Coverage:", coverage)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Coverage: 0.9990909090909091
Accuracy: 0.8398544131028207
Precision: 0.7259980807561524
Recall: 0.7107614607614607
F1 score: 0.7039231892022909


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print("Coverage:", round(coverage*100,2))
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Coverage: 99.91
Accuracy: 83.99
Precision: 72.6
Recall: 71.08
F1 score: 70.39
